In [ ]:
# pip install kafka-python

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [ ]:
Producer = KafkaProducer(
    bootstrap_servers=['54.83.103.179:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8'))   #we use this as def(value_serialzier(x): return dumps(x).encode('utf-8'))

In [ ]:
Producer.send('test', value='Okay, Nice to meet you!')
Producer.flush()
print('Message Sent!')

In [1]:
import requests, json, csv
from datetime import datetime
from kafka import KafkaProducer
from json import dumps
from time import sleep

# --- Kafka setup ---
producer = KafkaProducer(
    bootstrap_servers=['54.83.103.179:9092'],   # your EC2 public IP
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

# --- Endpoint ---
url = "https://graphql.coincap.io/"

# --- GraphQL query ---
query = """
{
  assets(first: 100) {
    edges {
      node {
        id
        symbol
        name
        rank
        priceUsd
        changePercent24Hr
        volumeUsd24Hr
        marketCapUsd
        supply
        vwapUsd24Hr
        website
        explorer
        logo
      }
    }
  }
}
"""

# --- Helper to handle floats safely ---
def safe_float(x):
    try:
        return float(x)
    except (TypeError, ValueError):
        return None

# --- Fetch function ---
def fetch_crypto_data():
    print("✅ Starting API call...", flush=True)
    response = requests.post(url, json={"query": query}, timeout=10)
    print("✅ API call complete, parsing JSON...", flush=True)

    if response.status_code != 200:
        raise Exception(f"API call failed: {response.status_code}\n{response.text}")

    data = response.json()
    assets = data["data"]["assets"]["edges"]
    cleaned = []
    for i in assets:
        a = i["node"]
        cleaned.append({
            "timestamp": datetime.utcnow().isoformat(),
            "id": a.get("id"),
            "symbol": a.get("symbol"),
            "name": a.get("name"),
            "rank": a.get("rank"),
            "priceUsd": safe_float(a.get("priceUsd")),
            "changePercent24Hr": safe_float(a.get("changePercent24Hr")),
            "volumeUsd24Hr": safe_float(a.get("volumeUsd24Hr")),
            "marketCapUsd": safe_float(a.get("marketCapUsd")),
            "supply": safe_float(a.get("supply")),
            "vwapUsd24Hr": safe_float(a.get("vwapUsd24Hr")),
            "website": a.get("website"),
            "explorer": a.get("explorer"),
            "logo": a.get("logo")
        })
    return cleaned

# --- Stream data continuously ---
while True:
    try:
        crypto_data = fetch_crypto_data()
        print(f"✅ Fetched {len(crypto_data)} crypto records.", flush=True)

        for record in crypto_data:
            print("✅ Sending to Kafka...", flush=True)
            producer.send('testing', value=record)
            print(f"📤 Sent to Kafka: {record['id']} at {record['timestamp']}", flush=True)

        print("✅ Flushing...", flush=True)
        producer.flush()
        print("✅ Done flushing.", flush=True)

        print("⏳ Waiting 10 seconds before next update...\n", flush=True)
        sleep(10)

    except requests.exceptions.Timeout:
        print("⚠️ API request timed out — retrying in 15s", flush=True)
        sleep(15)
    except KeyboardInterrupt:
        print("🛑 Stopped manually by user.")
        break
    except Exception as e:
        print(f"💥 Unexpected error: {e}", flush=True)
        sleep(15)


✅ Starting API call...
✅ API call complete, parsing JSON...
✅ Fetched 100 crypto records.
✅ Sending to Kafka...
📤 Sent to Kafka: bitcoin at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: ethereum at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: tether at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: xrp at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: binance-coin at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: solana at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: usd-coin at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: steth at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: tron at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: dogecoin at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: cardano at 2025-11-12T22:17:13.247413
✅ Sending to Kafka...
📤 Sent to Kafka: lido-finance-wsteth at 2025